In [145]:
import scipy.io
from scipy import sparse
import scipy.linalg
from scipy.sparse.linalg import inv

import pprint

import numpy as np

from matplotlib import pyplot as plt
import matplotlib

mat = scipy.io.loadmat(r"C:\Users\alaah\Desktop\Aer1513\AER1513_State_Estimation\Assignment_2\dataset2.mat")

#initialize all the data to variables
t = mat["t"]
x_true = mat["x_true"]
y_true = mat["y_true"]
th_true= mat["th_true"]
true_valid = mat["true_valid"]
landmarks_x_y = mat["l"]
r_l = mat["r"] 
r_var = mat["r_var"]
phi_l = mat["b"]
phi_var = mat["b_var"]
v = mat["v"]
v_var = mat["v_var"]
omega = mat["om"]
omega_var = mat["om_var"]
d = mat["d"]
x_c = mat["l"]

size = len(x_true)


In [ ]:
# Notes 
# Only compare the results of the estimator to the ground truth at indices where true_valid is equal to 1
# The distance d is the distance between the center  of the robot and the laser range finder and needs to be accounted for when conducting the estimation
# It is reasonable to assume guassian measurement noise for range and translational speed errors
# bearing and rotational speed errors are not very guassian and may be a cause of error for later calculations 
# all distribustions are unimodal 
# Can use EKF,
# Can use unicycle model of a mobile robot 
# non linear due to cos and sin
# d already accouted for in the bservation model 
# arctan for observation model 
# at every tims step we are gettting range measurments but the number of range measurments is changing (the y vector)
# can stack all the measurments toegther and treat i as high dimensional measurment but the length is variable 
# need to retune some parameters in the filter so that the error stays within the covariance network
# whenever you are comparing 2 angles make sure you go the short way around the circle (look for code that does that)


In [161]:
def motion_Jacob(T, theta_k_minus,v_k):
    F_k_minus = np.array([
        [1, 0, -T * np.sin(theta_k_minus) * v_k],
        [0, 1, T * np.cos(theta_k_minus) * v_k],
        [0, 0, 1]
    ])
    return F_k_minus

def motion_model(x_k_minus, y_k_minus, theta_k_minus, T, v_k, omega_k):
    f_ = np.array([
        x_k_minus + T * np.cos(theta_k_minus) * v_k,
        y_k_minus + T * np.sin(theta_k_minus) * v_k,
        theta_k_minus + omega_k
    ])
    return f_

def motion_model_noise(T, theta_k_minus):
    A = np.array([
        [T * np.cos(theta_k_minus), 0],
        [T * np.sin(theta_k_minus), 0],
        [0, 1]
    ])
    Q_k = np.array([
    [4.42e-3, 0],
    [0, 8.816e-3]
    ])
    Q_k_prime = A@Q_k@np.transpose(A)
    return Q_k_prime
    
    

def obsev_Jacob(x_k, y_k, theta_k, x_l, y_l, d = d[0][0]):
    G_k = np.zeros((2, 3))
    
    G_k[0, 0] = (x_k - x_l + d * np.cos(theta_k)) / np.sqrt((-x_k + x_l - d * np.cos(theta_k))**2 + (y_l - y_k - d * np.sin(theta_k))**2)
    G_k[0, 1] = (y_k - y_l + d * np.sin(theta_k)) / np.sqrt((-y_k + y_l - d * np.sin(theta_k))**2 + (x_l - x_k - d * np.cos(theta_k))**2)
    G_k[0, 2] = (d * ((x_l - x_k) * np.sin(theta_k) + (y_k - y_l) * np.cos(theta_k))) / np.sqrt((-d * np.sin(theta_k) + y_l - y_k)**2 + (-d * np.cos(theta_k) + x_l - x_k)**2)
    
    G_k[1, 0] = (y_l - y_k - d * np.sin(theta_k)) / ((-x_k + x_l - d * np.cos(theta_k))**2 + (y_l - y_k - d * np.sin(theta_k))**2)
    G_k[1, 1] = -1 * (x_l - x_k - d * np.cos(theta_k)) / ((y_k - y_l + d * np.sin(theta_k))**2 + (x_l - x_k - d * np.cos(theta_k))**2)
    G_k[1, 2] = (d * np.sin(theta_k) * (d * np.sin(theta_k) - y_l + y_k)) / ((d * np.sin(theta_k) - y_l + y_k)**2 + (-d * np.cos(theta_k) + x_l - x_k)**2) - (d * np.cos(theta_k) * (-d * np.cos(theta_k) + x_l - x_k)) / ((d * np.sin(theta_k) - y_l + y_k)**2 + (-d * np.cos(theta_k) + x_l - x_k)**2) - 1
    
    return G_k

def observ_model( x_k, y_k ,theta_k ,x_l, y_l, d = d[0][0]):
    
    r_k = (x_l - x_k - d * np.cos(theta_k))**2 + (y_l - y_k - d * np.sin(theta_k))**2

    phi_k = np.arctan2(y_l - y_k - d * np.sin(theta_k), x_l - x_k - d * np.cos(theta_k)) - theta_k
    phi_k = (phi_k + np.pi) % (2 * np.pi) - np.pi

    [g_1,g_2] = [r_k, phi_k]

    return [g_1,g_2]

def observ_model_noise(l):
    
    R_k = np.array([
    [9.003e-4, 0],
    [0, 6.714e-4]
    ])
    B_l = np.eye(l)
    
    R_k_l = np.kron(B_l, R_k)
    
    R_k_prime = R_k_l
    return R_k_prime



In [173]:

def Extended_Kalman_filter(x_hat_minus_initial, P_hat_minus_initial, r_max, v, omega,r_l,phi_l, time_step):
    x_hat_minus = x_hat_minus_initial
    P_hat_minus = P_hat_minus_initial


    x_pos = []
    y_pos = []
    theta_bearing = []

    x_pos_var = []
    y_pos_var = []
    theta_bearing_var = []

    for i in range(0,size):
        x_pos.append(x_hat_minus[0])
        y_pos.append(x_hat_minus[1])
        theta_bearing.append(x_hat_minus[2])

        x_pos_var.append(P_hat_minus[0,0])
        y_pos_var.append(P_hat_minus[1,1])
        theta_bearing_var.append(P_hat_minus[2,2])

        v_k = v[i][0]
        omega_k = omega[i][0]
        F_k_minus =motion_Jacob(time_step, x_hat_minus[2],v_k)

        Q_k_prime = motion_model_noise(time_step, x_hat_minus[2])

        P_check = F_k_minus@P_hat_minus@np.transpose(F_k_minus) + Q_k_prime
        x_check = motion_model(x_hat_minus[0],x_hat_minus[1],x_hat_minus[2], time_step, v_k, omega_k)

        r_k_l = r_l[i]
        phi_k_l = phi_l[i]
        y_k_l= []    
        l = 0
        landmarks = []
        for landmark_distance_index in range(0,len(r_k_l)):
            if r_k_l[landmark_distance_index]<r_max and r_k_l[landmark_distance_index]>0:
                l+=1
                landmarks.append(landmark_distance_index)
                
                y_k_l.append(np.array([[r_k_l[landmark_distance_index]],[phi_k_l[landmark_distance_index]]]))
        if l>0:
            y_k = np.concatenate(y_k_l, axis = 0).reshape(-1, 1)
            G_k_l = []
            for landmark in landmarks:
                G_K_l_temp = obsev_Jacob(x_check[0], x_check[1], x_check[2], landmarks_x_y[landmark][0], landmarks_x_y[landmark][1])
                G_k_l.append(G_K_l_temp)
            G_k = np.vstack(G_k_l)
            R_k_prime = observ_model_noise(l)
            G_k_transpose = np.transpose(G_k)
            temp_matrix = G_k@P_check@G_k_transpose + R_k_prime
            temp_matrix = np.linalg.inv(temp_matrix)
            K_k  = P_check@G_k_transpose@temp_matrix
            temp_matrix = K_k@G_k
            P_hat =(np.eye(3) - temp_matrix)@P_check
            x_pos_var = []
            y_pos_var = []
            theta_bearing_var = []
            
            
            g_ = []
            for landmark in landmarks:
                [g_1 g_2]= observ_model(x_check[0], x_check[0],  x_check[2], landmarks_x_y[landmark][0], landmarks_x_y[landmark][1])
                g_.append(g_1) 
                g_.append(g_2)  
            g_ = np.array(g_)
            
            diff = y_k - g_
            diff_mul = K_k@diff
            x_hat = 
            
            P_hat_minus = P_hat
            x_hat_minus = x_hat
            
        else:
            P_hat_minus = P_check
            x_hat_minus = x_check
            
    return x_pos, y_pos, theta_bearing, x_pos_var, y_pos_var, theta_bearing_var
        
        

SyntaxError: invalid syntax (3958327167.py, line 64)

In [ ]:
[x_pos, y_pos, theta_bearing, x_pos_var, y_pos_var, theta_bearing_var] =Extended_Kalman_filter([x_true[0][0], y_true[0][0], th_true[0][0]], np.diag([1,1,0.1]), 5, v, omega,r_l,phi_l,0.1)

[-2.910157363570845, -2.8835322699334203, -14.419734531194829, -860.6841048636228, 2995850.5927913133, 1454689866350.1328, 6.509807342875247e+24, -1.5756855807944482e+51, 2.0051195835544506e+103, -4.525548326397542e+205, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [178]:
x_hat_minus =[x_true[0][0], y_true[0][0], th_true[0][0]]
P_hat_minus = np.diag([1,1,0.1])
r_max = 5
time_step = 0.1

x_pos = []
y_pos = []
theta_bearing = []

x_pos_var = []
y_pos_var = []
theta_bearing_var = []

for i in range(0,size):
    x_pos.append(x_hat_minus[0])
    y_pos.append(x_hat_minus[1])
    theta_bearing.append(x_hat_minus[2])

    x_pos_var.append(P_hat_minus[0,0])
    y_pos_var.append(P_hat_minus[1,1])
    theta_bearing_var.append(P_hat_minus[2,2])

    v_k = v[i][0]
    omega_k = omega[i][0]
    F_k_minus =motion_Jacob(time_step, x_hat_minus[2],v_k)

    Q_k_prime = motion_model_noise(time_step, x_hat_minus[2])

    P_check = F_k_minus@P_hat_minus@np.transpose(F_k_minus) + Q_k_prime
    x_check = motion_model(x_hat_minus[0],x_hat_minus[1],x_hat_minus[2], time_step, v_k, omega_k)

    r_k_l = r_l[i]
    phi_k_l = phi_l[i]
    y_k_l= []    
    l = 0
    landmarks = []
    for landmark_distance_index in range(0,len(r_k_l)):
        if r_k_l[landmark_distance_index]<r_max and r_k_l[landmark_distance_index]>0:
            l+=1
            landmarks.append(landmark_distance_index)
            y_k_l.append(r_k_l[landmark_distance_index])
            y_k_l.append(phi_k_l[landmark_distance_index])
    if l>0:
        y_k = np.array(y_k_l)
        y_k = np.transpose(y_k)
        G_k_l = []
        for landmark in landmarks:
            G_K_l_temp = obsev_Jacob(x_check[0], x_check[1], x_check[2], landmarks_x_y[landmark][0], landmarks_x_y[landmark][1])
            G_k_l.append(G_K_l_temp)
        G_k = np.vstack(G_k_l)
        R_k_prime = observ_model_noise(l)
        G_k_transpose = np.transpose(G_k)
        temp_matrix = G_k@P_check@G_k_transpose + R_k_prime
        temp_matrix = np.linalg.inv(temp_matrix)
        K_k  = P_check@G_k_transpose@temp_matrix
        temp_matrix = K_k@G_k
        P_hat =(np.eye(3) - temp_matrix)@P_check
        x_pos_var = []
        y_pos_var = []
        theta_bearing_var = []
        
        
        g_ = []
        for landmark in landmarks:
            [g_1, g_2]= observ_model(x_check[0], x_check[0],  x_check[2], landmarks_x_y[landmark][0], landmarks_x_y[landmark][1])
            g_.append(g_1) 
            g_.append(g_2) 
        g_ = np.array(g_)
        g_ = np.transpose(g_)
        
        diff = y_k - g_
        diff_mul = K_k@diff
        
        x_hat = x_check + diff_mul
        P_hat_minus = P_hat
        x_hat_minus = x_hat
        
    else:
        P_hat_minus = P_check
        x_hat_minus = x_check

C:\Users\alaah\AppData\Local\Temp\ipykernel_30060\3973293406.py:35: RuntimeWarning: overflow encountered in double_scalars
  G_k[0, 0] = (x_k - x_l + d * np.cos(theta_k)) / np.sqrt((-x_k + x_l - d * np.cos(theta_k))**2 + (y_l - y_k - d * np.sin(theta_k))**2)
C:\Users\alaah\AppData\Local\Temp\ipykernel_30060\3973293406.py:36: RuntimeWarning: overflow encountered in double_scalars
  G_k[0, 1] = (y_k - y_l + d * np.sin(theta_k)) / np.sqrt((-y_k + y_l - d * np.sin(theta_k))**2 + (x_l - x_k - d * np.cos(theta_k))**2)
C:\Users\alaah\AppData\Local\Temp\ipykernel_30060\3973293406.py:37: RuntimeWarning: overflow encountered in double_scalars
  G_k[0, 2] = (d * ((x_l - x_k) * np.sin(theta_k) + (y_k - y_l) * np.cos(theta_k))) / np.sqrt((-d * np.sin(theta_k) + y_l - y_k)**2 + (-d * np.cos(theta_k) + x_l - x_k)**2)
C:\Users\alaah\AppData\Local\Temp\ipykernel_30060\3973293406.py:39: RuntimeWarning: overflow encountered in double_scalars
  G_k[1, 0] = (y_l - y_k - d * np.sin(theta_k)) / ((-x_k + x_l 

In [179]:
print(x_pos)

[3.019756132877692, -11.570482289772364, 116.6548451366422, -4382.17484675734, -3359506.837242708, 10663026324233.918, -1.1588214090613346e+26, 9.706281883030111e+51, -4.6084798482988496e+103, 1.3383597320500593e+207, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na